# Import libraries

In [1]:
%matplotlib inline

import datetime
import collections

import numpy as np
import ps_data_science_tools as ps_dst
from nn_modules import NetDropout
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

torch.set_printoptions(edgeitems=2)
torch.manual_seed(42)

# Dataset Loading and Preprocessing

In [2]:
data_path = '../cifar10_pytorch/data'
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']
class_names_dict = dict(zip(range(len(class_names)), class_names))

cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ]))

Files already downloaded and verified


In [3]:
# DataLoader 
dataloader = DataLoader(cifar10, batch_size=64, shuffle=False, num_workers=2)

# compute mean and std
mean, std = ps_dst.compute_mean_and_std(dataloader)

# save consts
dict_consts = {'mean': mean, 'std': std}
torch.save(dict_consts, 'dict_consts.pth')

# Out
print(f"Mean (RGB): {mean}")
print(f"Std Dev (RGB): {std}")

Mean (RGB): tensor([0.4914, 0.4822, 0.4465])
Std Dev (RGB): tensor([0.2470, 0.2435, 0.2616])


In [4]:
# loading with normalization
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]))
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]))

Files already downloaded and verified
Files already downloaded and verified


# Create model

In [5]:
# Device Configuration
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [6]:
# Loader Initialization
batch_size = 256
train_loader = DataLoader(cifar10, batch_size=batch_size,
                                           shuffle=True)
val_loader = DataLoader(cifar10_val, batch_size=batch_size,
                                         shuffle=False)

# Checking for NaN and Inf in batches
for images, labels in train_loader:
    has_nan_or_inf = torch.isnan(images).any() or torch.isinf(images).any()
    if has_nan_or_inf:
        print("Found NaN or Inf in batch!")

In [7]:
model = NetDropout(n_chans1=64).to(device=device)
optimizer = optim.Adam(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

ps_dst.training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    device = device
)
ps_dst.evaluate(model, train_loader, val_loader, device)

Training on device cuda.
2025-01-25 23:34:40.122028 Epoch 1, Training loss: 1.7295
2025-01-25 23:35:59.669631 Epoch 10, Training loss: 0.3545
2025-01-25 23:37:28.184442 Epoch 20, Training loss: 0.0869
2025-01-25 23:38:56.948197 Epoch 30, Training loss: 0.0559
2025-01-25 23:40:25.468630 Epoch 40, Training loss: 0.0477
2025-01-25 23:41:53.563204 Epoch 50, Training loss: 0.0369
2025-01-25 23:43:22.159383 Epoch 60, Training loss: 0.0316
2025-01-25 23:44:50.252934 Epoch 70, Training loss: 0.0273
2025-01-25 23:46:18.655337 Epoch 80, Training loss: 0.0281
2025-01-25 23:47:46.849180 Epoch 90, Training loss: 0.0245
2025-01-25 23:49:16.166705 Epoch 100, Training loss: 0.0222
Evaluating on device cuda.


{'train': {'accuracy': 0.995}, 'val': {'accuracy': 0.8111}}

model info

In [8]:
# count parameters
sum([p.numel() for p in model.parameters()])

1185290

save model

In [9]:
torch.save(model.state_dict(), 'model_' +'cifar10.pt')